# Init and aged vehicle DVA

This figure shows the evaluation of vehicle charging data from state of the art electric vehicles. \
The standard measurement enables the calaculation of an overall capacity and energy loss per vehicle. \
Also DVA is performed on the vehicle level through which degradation modes are assessed. Features of interest cannot always be derived from DVA, but if so, the can enable a deeper analysis of where the degradation takes place.

In [ ]:
import os
import sys
import math
sys.path.append(os.path.join(os.getcwd().partition('nature_soh')[0], "nature_soh"))
from src.config_base import GeneralConfig

In [ ]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
from matplotlib.ticker import FormatStrFormatter
import matplotlib.gridspec as gridspec
from matplotlib.ticker import AutoMinorLocator, FixedLocator
import matplotlib.patches as patches
import matplotlib.lines as mlines

In [ ]:
from src.data.read_feather import ReadFeather
from src.filtering.filter_methods import FilterMethods
from src.voltage_capacity_analysis.dva import DVA
from src.visualization.config_visualization import VisualizationConfig, instantiate_matplotlib_parameters, cm2inch
instantiate_matplotlib_parameters()
from src.visualization.colormaps import ColorMaps

In [ ]:
tum_orange_cmap = ColorMaps.orange_tum()
orange_colors = tum_orange_cmap(np.linspace(0,1, 10))
tum_orange_cmap

In [ ]:
tum_blue_cmap = ColorMaps.blue_tum()
blue_colors = tum_blue_cmap(np.linspace(0,1, 10))
tum_blue_cmap

In [ ]:
tum_cmap = ColorMaps.blue_orange_tum(grayscale=0.5)
colors = tum_cmap(np.linspace(0,1, 5))
tum_cmap

In [ ]:
def filter_func_preprocess(signal):
    perc_filter = 1/100 * len(signal) #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter * len(signal)))
    return filtered_signal

In [ ]:
read_feather = ReadFeather()
read_feather.set_filter_U(filter_func_preprocess)
read_feather.set_filter_Q(filter_func_preprocess)

## Load vehicle data

In [ ]:
%%capture
df_taycan =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"Taycan","Porsche_Taycan_JB_10A_CEE7_FTM_2023.feather"), lower_voltage=650, upper_voltage=830)
df_taycan_aged =  read_feather.read(os.path.join(GeneralConfig.path2data.value,"Taycan","Porsche_Taycan_JB_10A_CEE7_AMS_2024.feather"),lower_voltage=650, upper_voltage=830)

In [ ]:
%%capture
df_vw = read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_JB_8A_CEE7_FTM_2021.feather"), lower_voltage=360, upper_voltage=450)
df_vw_aged = read_feather.read(os.path.join(GeneralConfig.path2data.value,"VW","VW_ID3_JB_8A_CEE7_FTM_2024.feather"), lower_voltage=360, upper_voltage=450)

In [ ]:
%%capture
df_teslaY = read_feather.read(os.path.join(GeneralConfig.path2data.value,"Tesla","Tesla_Model_Y_JB_8A_CEE7_MI_2025.feather"), lower_voltage=300, upper_voltage=400)
df_teslaY_aged = read_feather.read(os.path.join(GeneralConfig.path2data.value,"Tesla","Tesla_Model_Y_JB_8A_CEE7_TW_2024.feather"), lower_voltage=300, upper_voltage=400)

In [ ]:
%%capture
df_tesla_aged = read_feather.read(os.path.join(GeneralConfig.path2data.value,"Tesla","Tesla_Model_3_JB_6A_CEE7_FTM_2025.feather"), lower_voltage=335, upper_voltage=365)

In [ ]:
## the sharp voltage increased of LFP cells can be filtered out, if the windowsize is too wide, as happens for the Tesla measurement from 2021
def filter_func_preprocess_tesla(signal):
    perc_filter = 0.1/100 * len(signal) #% filter
    filtered_signal = FilterMethods().rolling_mean_df(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter))
    #filtered_signal = FilterMethods().savgol(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter * len(signal)))
    return filtered_signal

In [ ]:
read_feather.set_filter_U(filter_func_preprocess_tesla)
read_feather.set_filter_Q(filter_func_preprocess_tesla)

In [ ]:
%%capture
df_tesla = read_feather.read(os.path.join(GeneralConfig.path2data.value,"Tesla","Tesla_Model_3_JB_6A_CEE7_FTM_2021.feather"), lower_voltage=335, upper_voltage=365)

### DVA

In [ ]:
def filter_func_v(signal):
    perc_filter = 1 #% filter
    filtered_signal = FilterMethods().rolling_mean_numpy(signal,window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    filtered_signal_reverse = FilterMethods().rolling_mean_numpy(np.flip(signal),window_size=FilterMethods().round_to_next_odd_number(perc_filter/100 * len(signal)))
    return (filtered_signal+np.flip(filtered_signal_reverse))/2

In [ ]:
dva_extractor = DVA()
dva_extractor.set_filter(filter_func_v)

In [ ]:
dva_taycan, cap_taycan = dva_extractor.get_dva(time=df_taycan["time_h"], capacity=df_taycan["Q"], voltage=df_taycan["U"],crop_cap_Ah=(0,200))
dva_taycan_aged, cap_taycan_aged = dva_extractor.get_dva(time=df_taycan_aged["time_h"], capacity=df_taycan_aged["Q"], voltage=df_taycan_aged["U"],crop_cap_Ah=(0,200))

In [ ]:
dva_vw, cap_vw = dva_extractor.get_dva(time=df_vw["time_h"], capacity=df_vw["Q"], voltage=df_vw["U"],crop_cap_Ah=(0,200))
dva_vw_aged, cap_vw_aged = dva_extractor.get_dva(time=df_vw_aged["time_h"], capacity=df_vw_aged["Q"], voltage=df_vw_aged["U"],crop_cap_Ah=(0,200))

In [ ]:
dva_tesla, cap_tesla = dva_extractor.get_dva(time=df_tesla["time_h"], capacity=df_tesla["Q"], voltage=df_tesla["U"],crop_cap_Ah=(1,220))
dva_tesla_aged, cap_tesla_aged = dva_extractor.get_dva(time=df_tesla_aged["time_h"], capacity=df_tesla_aged["Q"], voltage=df_tesla_aged["U"],crop_cap_Ah=(1,220))

In [ ]:
dva_teslaY, cap_teslaY = dva_extractor.get_dva(time=df_teslaY["time_h"], capacity=df_teslaY["Q"], voltage=df_teslaY["U"],crop_cap_Ah=(1,250))
dva_teslaY_aged, cap_teslaY_aged = dva_extractor.get_dva(time=df_teslaY_aged["time_h"], capacity=df_teslaY_aged["Q"], voltage=df_teslaY_aged["U"],crop_cap_Ah=(1,250))

## Figure

## POCV

In [ ]:
def plot_pocv_vw(axes):
    ax2 = axes.twiny()
    line1 = axes.plot(df_vw["Q"],df_vw["U"],color=VisualizationConfig.TUMblau.value,linewidth=1.5,label="$\ \ \ \ 400\,km$")
    line2 = axes.plot(df_vw_aged["Q"],df_vw_aged["U"],color=VisualizationConfig.TUMorange.value,linewidth=1.5,label="$40500\,km$")
    
    axes.set_ylabel(r"Vehicle voltage in V",fontsize=7)
    #axes.yaxis.set_label_coords(-0.05,0.5)
    #axes.set_xlabel("Charged capacity in Ah") 
    axes.set_xlim([0,150])
    tick_positions = np.array([0,50,100,150])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([25,75,125]))
    axes.set_ylim([360,450])
    tick_positions = np.array([360,390,420,450])
    axes.set_yticks(tick_positions)
    axes.tick_params('x', top=False, labeltop=False,bottom=True, labelbottom=False)
    axes.tick_params('y', left=True, labelleft=True,right=False, labelright=False)
    axes.grid()
    axes.set_axisbelow(True)
    # secondary x-axes 
    ax2.plot(df_vw["U"].mean()*(cap_vw)/1000,-10*np.ones(len(cap_vw)),color="white",linewidth=1.5)
    ax2.set_xlabel("Charged energy in kWh",fontsize=7)
    ax2.set_xlim(np.array([0,150]))
    tick_positions = np.array([0,50,100,150])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_vw.U.mean()/10000)*10:.0f}" for label in tick_positions])
    ax2.xaxis.set_minor_locator(FixedLocator([25,75,125]))
    # legend
    patch1 = mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='d', linestyle='None',
                          markersize=3, label='stage 2 shift')
    patch2 =  mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='o', linestyle='None',
                          markersize=3, label='H2 shift')
    axes.legend(handles=[line1[0], line2[0], patch1, patch2],loc="upper left",fontsize=5,ncols=2,handletextpad=0.4,handlelength=1.5,
               framealpha =1 , fancybox=False,labelspacing=0.2,columnspacing=0.7,edgecolor="black",title=r"$\bf{VW\ ID.3\ Pro\ Performance\ (C/NMC\,532)}$",title_fontsize=5,alignment='center')
    return axes

In [ ]:
def plot_pocv_taycan(axes):
    ax2 = axes.twiny()
    line1 = axes.plot(df_taycan["Q"],df_taycan["U"],color=VisualizationConfig.TUMblau.value,linewidth=1.5,label="$10000\,km$")
    line2 = axes.plot(df_taycan_aged["Q"],df_taycan_aged["U"],color=VisualizationConfig.TUMorange.value,linewidth=1.5,label="$70500\,km$")
    
    axes.set_ylabel(r"Vehicle voltage in V",fontsize=7)
    #axes.yaxis.set_label_coords(-0.05,0.5)
    #axes.set_xlabel("Charged capacity in Ah") 
    axes.set_xlim([0,120])
    tick_positions = np.array([0,40,80,120])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([20,60,100]))
    axes.set_ylim([650,830])
    tick_positions = np.array([650,695,740,785,830])
    axes.set_yticks(tick_positions)
    axes.tick_params('x', top=False, labeltop=False,bottom=True, labelbottom=False)
    axes.tick_params('y', left=True, labelleft=True,right=False, labelright=False)
    axes.grid()
    axes.set_axisbelow(True)
    # secondary x-axes 
    ax2.plot(df_taycan["U"].mean()*(cap_taycan)/1000,-10*np.ones(len(cap_taycan)),color="white",linewidth=1.5)
    ax2.set_xlabel("Charged energy in kWh",fontsize=7)
    ax2.set_xlim([0,120])
    tick_positions = np.array([0,40,80,120])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_taycan.U.mean()/10000)*10:.0f}" for label in tick_positions])
    ax2.xaxis.set_minor_locator(FixedLocator([20,60,100]))
    ax2.tick_params('x',which="both", top=True, labeltop=True,bottom=False, labelbottom=False)
    # legend
    patch1 = mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='d', linestyle='None',
                          markersize=3, label='stage 2 shift')
    #patch2 =  mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='o', linestyle='None',
    #                      markersize=3, label='H2 shift')
    axes.legend(handles=[line1[0], line2[0],patch1],loc="upper left",fontsize=5,ncols=2,handletextpad=0.4,handlelength=1.5,
               framealpha =1 , fancybox=False,labelspacing=0.2,columnspacing=0.7,edgecolor="black",title=r"$\bf{Porsche\ Taycan\ (C/NMC\,622)}$",title_fontsize=5,alignment='center')
    return axes

In [ ]:
def plot_pocv_tesla(axes):
    ax2 = axes.twiny()
    line1 = axes.plot(df_tesla["Q"],df_tesla["U"],color=VisualizationConfig.TUMblau.value,linewidth=1.5,label="$\ \ 1000\,km$")
    line2 = axes.plot(df_tesla_aged["Q"],df_tesla_aged["U"],color=VisualizationConfig.TUMorange.value,linewidth=1.5,label="$80100\,km$")
    
    axes.set_ylabel(r"Vehicle voltage in V",fontsize=7)
    #axes.yaxis.set_label_coords(-0.05,0.5)
    #axes.set_xlabel("Charged capacity in Ah") 
    axes.set_xlim([0,160])
    tick_positions = np.array([0,40,80,120,160])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([20,60,100,140]))
    axes.set_ylim([335,365])
    tick_positions = np.array([335,345,355,365])
    axes.set_yticks(tick_positions)
    axes.tick_params('x', top=False, labeltop=False,bottom=True, labelbottom=False)
    axes.tick_params('y', left=True, labelleft=True,right=False, labelright=False)
    axes.grid()
    axes.set_axisbelow(True)
    # secondary x-axes 
    ax2.plot(df_tesla["U"].mean()*(df_tesla)/1000,-10*np.ones(len(df_tesla)),color="white",linewidth=1.5)
    ax2.set_xlabel("Charged energy in kWh",fontsize=7)
    ax2.set_xlim([0,160])
    tick_positions = np.array([0,40,80,120,160])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_tesla.U.mean()/1000)*1:.0f}" for label in tick_positions[:-1]]+[56])
    ax2.xaxis.set_minor_locator(FixedLocator([20,60,100,140]))
    ax2.tick_params('x',which="both", top=True, labeltop=True,bottom=False, labelbottom=False)
    # legend
    patch1 = mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='d', linestyle='None',
                          markersize=3, label='stage 2 shift')
    #patch2 =  mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='o', linestyle='None',
    #                      markersize=3, label='H2 shift')
    axes.legend(handles=[line1[0], line2[0],patch1],loc="upper left",fontsize=5,ncols=2,handletextpad=0.4,handlelength=1.5,
               framealpha =1 , fancybox=False,labelspacing=0.05,columnspacing=0.7,edgecolor="black",title=r"$\bf{Tesla\ Model\ 3\ SR+\ (C/LFP)}$",title_fontsize=5,alignment='center')
    return axes

In [ ]:
def plot_pocv_teslaY(axes):
    ax2 = axes.twiny()
    line1 = axes.plot(df_teslaY["Q"],df_teslaY["U"],color=VisualizationConfig.TUMblau.value,linewidth=1.5,label="$24800\,km$")
    line2 = axes.plot(df_teslaY_aged["Q"],df_teslaY_aged["U"],color=VisualizationConfig.TUMorange.value,linewidth=1.5,label="$77600\,km$")
    
    axes.set_ylabel(r"Vehicle voltage in V",fontsize=7)
    #axes.yaxis.set_label_coords(-0.05,0.5)
    #axes.set_xlabel("Charged capacity in Ah") 
    axes.set_xlim([0,220])
    tick_positions = np.array([0,55,110,165,220])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([22.5,77.5,132.5,187.5]))
    axes.set_ylim([300,400])
    tick_positions = np.array([300,325,350,375,400])
    axes.set_yticks(tick_positions)
    axes.tick_params('x', top=False, labeltop=False,bottom=True, labelbottom=False)
    axes.tick_params('y', left=True, labelleft=True,right=False, labelright=False)
    axes.grid()
    axes.set_axisbelow(True)
    # secondary x-axes 
    ax2.plot(df_tesla["U"].mean()*(df_teslaY)/1000,-10*np.ones(len(df_teslaY)),color="white",linewidth=1.5)
    ax2.set_xlabel("Charged energy in kWh",fontsize=7)
    ax2.set_xlim([0,220])
    tick_positions = np.array([0,55,110,165,220])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_teslaY.U.mean()/10000)*10:.0f}" for label in tick_positions])
    ax2.xaxis.set_minor_locator(FixedLocator([22.5,77.5,132.5,187.5]))
    ax2.tick_params('x',which="both", top=True, labeltop=True,bottom=False, labelbottom=False)
    #legend
    patch1 = mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='d', linestyle='None',
                          markersize=3, label='stage 2 shift')
    patch2 =  mlines.Line2D([], [], color='gray', markeredgecolor="k", marker='o', linestyle='None',
                          markersize=3, label='H2 shift')
    axes.legend(handles=[line1[0], line2[0], patch1, patch2],loc="upper left",fontsize=5,ncols=2,handletextpad=0.4,handlelength=1.5,
               framealpha =1 , fancybox=False,labelspacing=0.2,columnspacing=0.7,edgecolor="black",title=r"$\bf{Tesla\ Model\ Y\ LR\ (C/NMC\,811)}$",title_fontsize=5,alignment='center')
    return axes

## Features of interest pOCV

In [ ]:
def add_FOI_vw_pocv(axes):
    deltaQ = df_vw.Q.max() - df_vw_aged.Q.max()
    deltaE = df_vw.E.max() - df_vw_aged.E.max()
    rectangle = patches.Rectangle((df_vw_aged.Q.max(),360), deltaQ, 450-360,fc="lightgray",ec="lightgray",alpha=0.7)
    axes.add_artist(rectangle)
    axes.annotate(f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$ | $\Delta E$ = {deltaE:.1f}$\,kWh$',
                  (0.6, 0.02), xycoords=rectangle,rotation=90, annotation_clip=True,fontsize=5,
                 horizontalalignment = "center", verticalalignment = "bottom")
    #axes.annotate(text='', xy=(145,440), xytext=(115,440), arrowprops=dict(arrowstyle='<|-', shrinkA=0, shrinkB=0,facecolor="k"))
    #axes.annotate(text=f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$'+'\n'+f'$\Delta E$ = {deltaE:.1f}$\,kWh$', xy=(150,420),horizontalalignment="right",verticalalignment="bottom",fontsize=5)#,bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.7))
    return axes

In [ ]:
def add_FOI_taycan_pocv(axes):
    deltaQ = df_taycan.Q.max() - df_taycan_aged.Q.max()
    deltaE = df_taycan.E.max() - df_taycan_aged.E.max()
    rectangle = patches.Rectangle((df_taycan_aged.Q.max(),650), deltaQ, 830-650,fc="lightgray",ec="lightgray",alpha=0.7)
    axes.add_artist(rectangle)
    axes.annotate(f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$ | $\Delta E$ = {deltaE:.1f}$\,kWh$',
                  (0.6, 0.02), xycoords=rectangle,rotation=90, annotation_clip=True,fontsize=5,
                 horizontalalignment = "center", verticalalignment = "bottom")
    # axes.annotate(text='', xy=(115,820), xytext=(90,820), arrowprops=dict(arrowstyle='<|-', shrinkA=0, shrinkB=0,facecolor="k"))
    # axes.annotate(text=f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$'+'\n'+f'$\Delta E$ = {deltaE:.1f}$\,kWh$', xy=(120,785),horizontalalignment="right",verticalalignment="bottom",fontsize=5)#,bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.7))
    return axes

In [ ]:
def add_FOI_tesla_pocv(axes):
    deltaQ = df_tesla.Q.max() - df_tesla_aged.Q.max()
    deltaE = df_tesla.E.max() - df_tesla_aged.E.max()
    rectangle = patches.Rectangle((df_tesla_aged.Q.max(),335), deltaQ, 365-335,fc="lightgray",ec="lightgray",alpha=0.7)
    axes.add_artist(rectangle)
    axes.annotate(f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$ | $\Delta E$ = {deltaE:.1f}$\,kWh$',
                  (0.6, 0.02), xycoords=rectangle,rotation=90, annotation_clip=True,fontsize=5,
                 horizontalalignment = "center", verticalalignment = "bottom",bbox=dict(boxstyle='square,pad=0',fc='lightgray', ec='none', alpha=0.6))
    # axes.annotate(text='', xy=(159,363), xytext=(140,363), arrowprops=dict(arrowstyle='<|-', shrinkA=0, shrinkB=0,facecolor="k"))
    # axes.annotate(text=f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$'+'\n'+f'$\Delta E$ = {deltaE:.1f}$\,kWh$', xy=(121,357.5),horizontalalignment="left",verticalalignment="bottom",fontsize=5)#,bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.7))
    return axes

In [ ]:
def add_FOI_teslaY_pocv(axes):
    deltaQ =  df_teslaY.Q.max() - df_teslaY_aged.Q.max() #
    deltaE = df_teslaY.E.max() - df_teslaY_aged.E.max()
    rectangle = patches.Rectangle((df_teslaY_aged.Q.max(),300), deltaQ+1, 400-300,fc="lightgray",ec="lightgray",alpha=0.7)
    axes.add_artist(rectangle)
    axes.annotate(f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$ | $\Delta E$ = {deltaE:.1f}$\,kWh$',
                  (0.6, 0.02), xycoords=rectangle,rotation=90, annotation_clip=True,fontsize=5,
                 horizontalalignment = "center", verticalalignment = "bottom")
    # axes.annotate(text='', xy=(195,395), xytext=(155,395), arrowprops=dict(arrowstyle='<|-', shrinkA=0, shrinkB=0,facecolor="k"))
    # axes.annotate(text=f'$\Delta Q$ = {deltaQ:.1f}$\,Ah$'+'\n'+f'$\Delta E$ = {deltaE:.1f}$\,kWh$', xy=(200,375),horizontalalignment="right",verticalalignment="bottom",fontsize=5)#,bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.7))
    return axes

## DVA

In [ ]:
def plot_dva_vw(axes):
    ax2 = axes.twiny()
    axes.plot(cap_vw,dva_vw*2*78,color=VisualizationConfig.TUMblau.value,linewidth=1.5)
    axes.plot(cap_vw_aged,dva_vw_aged*2*78,color=VisualizationConfig.TUMorange.value,linewidth=1.5)
    
    axes.set_ylabel(r"$Q_{N}\cdot \frac{dU}{dQ}$ in V",fontsize=7)
    #axes.yaxis.set_label_coords(-0.05,0.5)
    axes.set_xlabel("Charged capacity in Ah",fontsize=7) 
    axes.set_xlim([0,150])
    tick_positions = np.array([0,50,100,150])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([25,75,125]))
    axes.set_ylim([0,216])
    tick_positions = np.array([0,51,108,108+108/2,216])
    axes.set_yticks(tick_positions)
    #axes.yaxis.set_minor_locator(FixedLocator([54,108+108/2]))
    axes.tick_params('x', top=False, labeltop=False,bottom=True, labelbottom=True)
    axes.tick_params('y', left=True, labelleft=True,right=False, labelright=False)
    axes.grid()
    axes.set_axisbelow(True) 
    # secondary x-axes 
    ax2.plot(df_vw["U"].mean()*(cap_vw)/1000,-10*np.ones(len(cap_vw)),color="white",linewidth=1.5)
    #ax2.set_xlabel("Charged energy in kWh")
    ax2.set_xlim(np.array([0,150]))
    tick_positions = np.array([0,50,100,150])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_vw.U.mean()/10000)*10:.0f}" for label in tick_positions])
    ax2.xaxis.set_minor_locator(FixedLocator([25,75,125]))
    ax2.tick_params('x', top=True, labeltop=False,bottom=True, labelbottom=False)
    return axes

In [ ]:
def plot_dva_taycan(axes):
    ax2 = axes.twiny()
    axes.plot(cap_taycan,dva_taycan*2*56,color=VisualizationConfig.TUMblau.value,linewidth=1.5)
    axes.plot(cap_taycan_aged,dva_taycan_aged*2*56,color=VisualizationConfig.TUMorange.value,linewidth=1.5)
    
    axes.set_ylabel(r"$Q_{N}\cdot \frac{dU}{dQ}$ in V")
    #axes.yaxis.set_label_coords(-0.05,0.5)
    axes.set_xlabel("Charged capacity in Ah") 
    axes.set_xlim([0,120])
    tick_positions = np.array([0,40,80,120])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([20,60,100]))
    axes.set_ylim([0,198*2])
    tick_positions = np.array([0,99,198,297,396])
    axes.set_yticks(tick_positions)
    #axes.yaxis.set_minor_locator(FixedLocator([99,297]))
    axes.tick_params('x',which="both", top=False, labeltop=False,bottom=True, labelbottom=True)
    #axes.yaxis.set_minor_locator(FixedLocator([385,435]))
    axes.grid()
    axes.set_axisbelow(True)
    # secondary x-axes 
    ax2.plot(df_taycan["U"].mean()*(cap_taycan)/1000,-10*np.ones(len(cap_taycan)),color="white",linewidth=1.5)
    #ax2.set_xlabel("Charged energy in kWh",fontsize=7)
    ax2.set_xlim(np.array([0,120]))
    tick_positions =  np.array([0,40,80,120])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_taycan.U.mean()/10000)*10:.0f}" for label in tick_positions])
    ax2.xaxis.set_minor_locator(FixedLocator([20,60,100]))
    ax2.tick_params('x',which="both", top=True, labeltop=False,bottom=False, labelbottom=False)
    return axes

In [ ]:
def plot_dva_tesla(axes):
    ax2 = axes.twiny()
    axes.plot(cap_tesla,dva_tesla*161,color=VisualizationConfig.TUMblau.value,linewidth=1.5)
    axes.plot(cap_tesla_aged,dva_tesla_aged*161,color=VisualizationConfig.TUMorange.value,linewidth=1.5)
    
    axes.set_ylabel(r"$Q_{N}\cdot \frac{dU}{dQ}$ in V")
    #axes.yaxis.set_label_coords(-0.05,0.5)
    axes.set_xlabel("Charged capacity in Ah") 
    axes.set_xlim([0,160])
    tick_positions = np.array([0,40,80,120,160])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([20,60,100,140]))
    axes.set_ylim([0,106*2])
    tick_positions = np.array([0,53,106,159,212])
    axes.set_yticks(tick_positions)
    #axes.set_ylim([0,106])
    #tick_positions = np.array([0,53/2,53,53+53/2,106])
    #axes.set_yticks(tick_positions)
    #axes.yaxis.set_minor_locator(FixedLocator([53/2,53+53/2]))
    axes.grid()
    axes.set_axisbelow(True)
    # secondary x-axes 
    ax2.plot(df_tesla["U"].mean()*(cap_tesla)/1000,-10*np.ones(len(cap_tesla)),color="white",linewidth=1.5)
   # ax2.set_xlabel("Charged energy in kWh")
    ax2.set_xlim([0,160])
    tick_positions = np.array([0,40,80,120,160])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_tesla.U.mean()/1000)*1:.0f}" for label in tick_positions])
    ax2.xaxis.set_minor_locator(FixedLocator([20,60,100,140]))
    ax2.tick_params('x',which="both", top=True, labeltop=False,bottom=False, labelbottom=False)
    return axes

In [ ]:
def plot_dva_teslaY(axes):
    ax2 = axes.twiny()
    axes.plot(cap_teslaY,dva_teslaY*5*46,color=VisualizationConfig.TUMblau.value,linewidth=1.5)
    axes.plot(cap_teslaY_aged,dva_teslaY_aged*5*46,color=VisualizationConfig.TUMorange.value,linewidth=1.5)
    
    axes.set_ylabel(r"$Q_{N}\cdot \frac{dU}{dQ}$ in V")
    #axes.yaxis.set_label_coords(-0.05,0.5)
    axes.set_xlabel("Charged capacity in Ah") 
    axes.set_xlim([0,220])
    tick_positions = np.array([0,55,110,165,220])
    axes.set_xticks(tick_positions)
    axes.xaxis.set_minor_locator(FixedLocator([22.5,77.5,132.5,187.5]))
    axes.set_ylim([0,96*2])
    tick_positions = np.array([0,48,48*2,144,96*2])
    axes.set_yticks(tick_positions)
    #axes.yaxis.set_minor_locator(FixedLocator([48,144]))
    axes.grid()
    axes.set_axisbelow(True)
    # secondary x-axes 
    ax2.plot(df_tesla["U"].mean()*(cap_teslaY)/1000,-10*np.ones(len(cap_teslaY)),color="white",linewidth=1.5)
    # ax2.set_xlabel("Charged energy in kWh")
    ax2.set_xlim([0,220])
    tick_positions = np.array([0,55,110,165,220])
    ax2.set_xticks(tick_positions)
    ax2.set_xticklabels([f"{round(label*df_teslaY.U.mean()/10000)*10:.0f}" for label in tick_positions])
    ax2.xaxis.set_minor_locator(FixedLocator([22.5,77.5,132.5,187.5]))
    ax2.tick_params('x',which="both", top=True, labeltop=False,bottom=False, labelbottom=False)
    return axes

## Degradation Mode assessment

In [ ]:
def DMs_vw_dva():
    print(f"Energy I: {np.max(df_vw.E):.2f}")
    print(f"Energy II: {np.max(df_vw_aged.E):.2f}")
    total_cap_loss = np.max(cap_vw_aged)-np.max(cap_vw)
    print(f"Capacity I: {np.max(cap_vw):.2f}")
    print(f"Capacity II: {np.max(cap_vw_aged):.2f}")
    print(f"Total capacity loss = {total_cap_loss:.2f}")
    #anode
    stage_2_init_dva =np.max(dva_vw[cap_vw>80])
    stage_2_init_cap =cap_vw[dva_vw==stage_2_init_dva]
    
    stage_2_aged_dva =np.max(dva_vw_aged[cap_vw_aged>80])
    stage_2_aged_cap =cap_vw_aged[dva_vw_aged==stage_2_aged_dva]

    delta_stage_2 = stage_2_aged_cap.values[0]-stage_2_init_cap.values[0]
    print(f"Q_NE I: {stage_2_init_cap.values[0]:.2f}")
    print(f"Q_NE II: {stage_2_aged_cap.values[0]:.2f}")
    print(f"LAM_NE = {delta_stage_2:.2f} Ah")
    # cathode
    c_init_dva =np.max(dva_vw[(cap_vw>120)&(cap_vw<140)])
    c_init_cap =cap_vw[dva_vw==c_init_dva]
    
    c_aged_dva =np.max(dva_vw_aged[(cap_vw_aged>120)&(cap_vw_aged<130)])
    c_aged_cap =cap_vw_aged[dva_vw_aged==c_aged_dva]
    delta_H_2 = (np.max(cap_vw_aged)-c_aged_cap.values[0])-(np.max(cap_vw)-c_init_cap.values[0])
    print(f"Q_PE I: {(np.max(cap_vw)-c_init_cap.values[0]):.2f}")
    print(f"Q_PE II: {(np.max(cap_vw_aged)-c_aged_cap.values[0]):.2f}")
    print(f"LAM_PE = {delta_H_2:.2f}")
    # LLI
    dQB =  (np.max(cap_vw_aged)-stage_2_aged_cap.values[0])-(np.max(cap_vw)-stage_2_init_cap.values[0])
    print(f"Q_B I: {(np.max(cap_vw)-stage_2_init_cap.values[0]):.2f}")
    print(f"Q_B II: {(np.max(cap_vw_aged)-stage_2_aged_cap.values[0]):.2f}")
    print(f"dQB= {dQB:.2f}")
    delta_LLI = dQB - delta_H_2
    print(f"LLI = {delta_LLI:.2f}")
    return {"stage_2_init_dva":stage_2_init_dva,
           "stage_2_init_cap":stage_2_init_cap,
           "stage_2_aged_dva":stage_2_aged_dva,
           "stage_2_aged_cap":stage_2_aged_cap,
           "c_init_dva":c_init_dva,
           "c_init_cap":c_init_cap,
           "c_aged_dva":c_aged_dva,
           "c_aged_cap":c_aged_cap}
scatter_dict_vw=DMs_vw_dva()

In [ ]:
def DMs_taycan_dva():
    print(f"Energy I: {np.max(df_taycan.E):.2f}")
    print(f"Energy II: {np.max(df_taycan_aged.E):.2f}")
    total_cap_loss = np.max(cap_taycan_aged)-np.max(cap_taycan)
    print(f"Capacity I: {np.max(cap_taycan):.2f}")
    print(f"Capacity II: {np.max(cap_taycan_aged):.2f}")
    print(f"Total capacity loss = {total_cap_loss:.2f}")
    #anode
    stage_2_init_dva =np.max(dva_taycan[cap_taycan>60])
    stage_2_init_cap =cap_taycan[dva_taycan==stage_2_init_dva]
    
    stage_2_aged_dva =np.max(dva_taycan_aged[cap_taycan_aged>60])
    stage_2_aged_cap =cap_taycan_aged[dva_taycan_aged==stage_2_aged_dva]

    delta_stage_2 = stage_2_aged_cap.values[0]-stage_2_init_cap.values[0]
    print(f"Q_NE I: {stage_2_init_cap.values[0]:.2f}")
    print(f"Q_NE II: {stage_2_aged_cap.values[0]:.2f}")
    print(f"LAM_NE = {delta_stage_2:.2f} Ah")
    # cathode # cannot be determined
    c_init_dva = 0 #np.max(dva_taycan[(cap_taycan>80)&(cap_taycan<100)])
    c_init_cap = 0 #cap_taycan[dva_taycan==c_init_dva]
    
    c_aged_dva = 0 #np.max(dva_taycan_aged[(cap_taycan_aged>80)&(cap_taycan_aged<100)])
    c_aged_cap = 0 #cap_taycan_aged[dva_taycan_aged==c_aged_dva]
    delta_H_2 = 0#(np.max(cap_taycan_aged)-c_aged_cap.values[0])-(np.max(cap_taycan)-c_init_cap.values[0])
    print(f"Q_PE I: {0}")
    print(f"Q_PE II: {0}")
    print(f"LAM_PE = {delta_H_2:.2f}")
    # LLI
    dQB =  (np.max(cap_taycan_aged)-stage_2_aged_cap.values[0])-(np.max(cap_taycan)-stage_2_init_cap.values[0])
    print(f"Q_B I: {(np.max(cap_taycan)-stage_2_init_cap.values[0]):.2f}")
    print(f"Q_B II: {(np.max(cap_taycan_aged)-stage_2_aged_cap.values[0]):.2f}")
    print(f"dQB= {dQB:.2f}")
    delta_LLI = dQB - delta_H_2
    print(f"LLI = {delta_LLI:.2f}")
    return {"stage_2_init_dva":stage_2_init_dva,
           "stage_2_init_cap":stage_2_init_cap,
           "stage_2_aged_dva":stage_2_aged_dva,
           "stage_2_aged_cap":stage_2_aged_cap,
           "c_init_dva":c_init_dva,
           "c_init_cap":c_init_cap,
           "c_aged_dva":c_aged_dva,
           "c_aged_cap":c_aged_cap}
scatter_dict_taycan = DMs_taycan_dva()

In [ ]:
def DMs_tesla_dva():
    print(f"Energy I: {np.max(df_tesla.E):.2f}")
    print(f"Energy II: {np.max(df_tesla_aged.E):.2f}")
    total_cap_loss = np.max(cap_tesla_aged)-np.max(cap_tesla)
    print(f"Capacity I: {np.max(cap_tesla):.2f}")
    print(f"Capacity II: {np.max(cap_tesla_aged):.2f}")
    print(f"Total capacity loss = {total_cap_loss:.2f}")
    #anode
    stage_2_init_dva =np.max(dva_tesla[(cap_tesla>80)&(cap_tesla<100)])
    stage_2_init_cap =cap_tesla[dva_tesla==stage_2_init_dva]
    
    stage_2_aged_dva =np.max(dva_tesla_aged[(cap_tesla_aged>80)&(cap_tesla_aged<100)])
    stage_2_aged_cap =cap_tesla_aged[dva_tesla_aged==stage_2_aged_dva]

    delta_stage_2 = stage_2_aged_cap.values[0]-stage_2_init_cap.values[0]
    print(f"Q_NE I: {stage_2_init_cap.values[0]:.2f}")
    print(f"Q_NE II: {stage_2_aged_cap.values[0]:.2f}")
    print(f"LAM_NE = {delta_stage_2:.2f} Ah")
    # cathode # no PE feature for LFP cells
    c_init_dva = 0 #np.max(dva_tesla[(cap_tesla>80)&(cap_tesla<100)])
    c_init_cap = 0 #cap_tesla[dva_tesla==c_init_dva]
    
    c_aged_dva = 0 #np.max(dva_tesla_aged[(cap_tesla_aged>80)&(cap_tesla_aged<100)])
    c_aged_cap = 0 #cap_tesla_aged[dva_tesla_aged==c_aged_dva]
    delta_H_2 = 0 #(np.max(cap_tesla_aged)-c_aged_cap.values[0])-(np.max(cap_tesla)-c_init_cap.values[0])
    print(f"Q_PE I: {0}")
    print(f"Q_PE II: {0}")
    print(f"LAM_PE = {delta_H_2:.2f}")
    # LLI
    dQB =  (np.max(cap_tesla_aged)-stage_2_aged_cap.values[0])-(np.max(cap_tesla)-stage_2_init_cap.values[0])
    print(f"Q_B I: {(np.max(cap_tesla)-stage_2_init_cap.values[0]):.2f}")
    print(f"Q_B II: {(np.max(cap_tesla_aged)-stage_2_aged_cap.values[0]):.2f}")
    print(f"dQB= {dQB:.2f}")
    delta_LLI = dQB - delta_H_2
    print(f"LLI = {delta_LLI:.2f}")
    return {"stage_2_init_dva":stage_2_init_dva,
           "stage_2_init_cap":stage_2_init_cap,
           "stage_2_aged_dva":stage_2_aged_dva,
           "stage_2_aged_cap":stage_2_aged_cap,
           "c_init_dva":c_init_dva,
           "c_init_cap":c_init_cap,
           "c_aged_dva":c_aged_dva,
           "c_aged_cap":c_aged_cap}
scatter_dict_tesla = DMs_tesla_dva()

In [ ]:
def DMs_teslaY_dva():
    print(f"Energy I: {np.max(df_teslaY.E):.2f}")
    print(f"Energy II: {np.max(df_teslaY_aged.E):.2f}")
    total_cap_loss = np.max(cap_teslaY_aged)-np.max(cap_teslaY)
    print(f"Capacity I: {np.max(cap_teslaY):.2f}")
    print(f"Capacity II: {np.max(cap_teslaY_aged):.2f}")
    print(f"Total capacity loss = {total_cap_loss:.2f}")
    #anode
    stage_2_init_dva =np.max(dva_teslaY[(cap_teslaY>110)&(cap_teslaY<130)])
    stage_2_init_cap =cap_teslaY[dva_teslaY==stage_2_init_dva]
    
    stage_2_aged_dva =np.max(dva_teslaY_aged[(cap_teslaY_aged>110)&(cap_teslaY_aged<130)])
    stage_2_aged_cap =cap_teslaY_aged[dva_teslaY_aged==stage_2_aged_dva]

    delta_stage_2 = stage_2_aged_cap.values[0]-stage_2_init_cap.values[0]
    print(f"Q_NE I: {stage_2_init_cap.values[0]:.2f}")
    print(f"Q_NE II: {stage_2_aged_cap.values[0]:.2f}")
    print(f"LAM_NE = {delta_stage_2:.2f} Ah")
    #cathode
    c_init_dva =np.max(dva_teslaY[(cap_teslaY>130)&(cap_teslaY<165)])
    c_init_cap =cap_teslaY[dva_teslaY==c_init_dva]
    
    c_aged_dva =np.max(dva_teslaY_aged[(cap_teslaY_aged>130)&(cap_teslaY_aged<165)])
    c_aged_cap =cap_teslaY_aged[dva_teslaY_aged==c_aged_dva]
    delta_H_2 = (np.max(cap_teslaY_aged)-c_aged_cap.values[0])-(np.max(cap_teslaY)-c_init_cap.values[0])
    print(f"Q_PE I: {(np.max(cap_teslaY)-c_init_cap.values[0]):.2f}")
    print(f"Q_PE II: {(np.max(cap_teslaY_aged)-c_aged_cap.values[0]):.2f}")
    print(f"LAM_PE = {delta_H_2:.2f}")
    # LLI
    dQB =  (np.max(cap_teslaY_aged)-stage_2_aged_cap.values[0])-(np.max(cap_teslaY)-stage_2_init_cap.values[0])
    print(f"Q_B I: {(np.max(cap_teslaY)-stage_2_init_cap.values[0]):.2f}")
    print(f"Q_B II: {(np.max(cap_teslaY_aged)-stage_2_aged_cap.values[0]):.2f}")
    print(f"dQB= {dQB:.2f}")
    delta_LLI = dQB - delta_H_2
    print(f"LLI = {delta_LLI:.2f}")
    return {"stage_2_init_dva":stage_2_init_dva,
           "stage_2_init_cap":stage_2_init_cap,
           "stage_2_aged_dva":stage_2_aged_dva,
           "stage_2_aged_cap":stage_2_aged_cap,
           "c_init_dva":c_init_dva,
           "c_init_cap":c_init_cap,
           "c_aged_dva":c_aged_dva,
           "c_aged_cap":c_aged_cap}
scatter_dict_teslaY = DMs_teslaY_dva()

## Features of interest DVA

In [ ]:
def add_FOI_vw_dva(axes, scatter_dict):
    Q_nom = 2*78
    #anode  
    axes.scatter(scatter_dict["stage_2_init_cap"],scatter_dict["stage_2_init_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMblau.value,s=10,zorder=5,ec="k")
    axes.scatter(scatter_dict["stage_2_aged_cap"],scatter_dict["stage_2_aged_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMorange.value,s=10,zorder=5,ec="k")
    
    axes.annotate(text='', xy=(scatter_dict["stage_2_init_cap"]+5,170), xytext=(scatter_dict["stage_2_init_cap"]-5,170),fontsize=9,
                      arrowprops=dict(arrowstyle='->', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{NE}=0.2\,Ah$', xy=(scatter_dict["stage_2_init_cap"]-2,180),fontsize=5,ha="center")
    # cathode
    axes.scatter(scatter_dict["c_init_cap"],scatter_dict["c_init_dva"]*Q_nom,marker="o",color=VisualizationConfig.TUMblau.value,s=10,zorder=5,ec="k")
    axes.scatter(scatter_dict["c_aged_cap"],scatter_dict["c_aged_dva"]*Q_nom,marker="o",color=VisualizationConfig.TUMorange.value,s=10,zorder=5,ec="k")
    
    axes.annotate(text='', xy=(scatter_dict["c_aged_cap"]+5+2,140), xytext=(scatter_dict["c_aged_cap"]-5+2,140),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{PE}=1.3\,Ah$', xy=(scatter_dict["c_aged_cap"]+2,150+2),fontsize=5,ha="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.6))
    # LLI
    axes.annotate(text='', xy=(np.max(cap_vw_aged)+5+2,95), xytext=(np.max(cap_vw_aged)-5+2,95),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{B}=6.3\,Ah$', xy=(np.max(cap_vw_aged)-2,73),fontsize=5,ha="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.6))
    # m phase shift
    axes.annotate(text='', xy=(80,55), xytext=(67,104), arrowprops=dict(arrowstyle='<|-', shrinkA=0, shrinkB=0,facecolor="k"))
    axes.annotate(text='M shift', xy=(83,53),horizontalalignment="left",verticalalignment="bottom",fontsize=5)
    return axes

In [ ]:
def add_FOI_taycan_dva(axes,scatter_dict):
    Q_nom = 2*56
    #anode  
    axes.scatter(scatter_dict["stage_2_init_cap"],scatter_dict["stage_2_init_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMblau.value,s=10,zorder=5,ec="k")
    axes.scatter(scatter_dict["stage_2_aged_cap"],scatter_dict["stage_2_aged_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMorange.value,s=10,zorder=5,ec="k")
    
    axes.annotate(text='', xy=(scatter_dict["stage_2_init_cap"]+4,310), xytext=(scatter_dict["stage_2_init_cap"]-4,310),fontsize=9,
                      arrowprops=dict(arrowstyle='->', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{NE}=0.3\,Ah$', xy=(scatter_dict["stage_2_init_cap"],330),fontsize=5,ha="center")
    # cathode
    # axes.scatter(scatter_dict["c_init_cap"],scatter_dict["c_init_dva"]*Q_nom,marker="o",color=VisualizationConfig.TUMblau.value,s=10,zorder=5,ec="k")
    # axes.scatter(scatter_dict["c_aged_cap"],scatter_dict["c_aged_dva"]*Q_nom,marker="o",color=VisualizationConfig.TUMorange.value,s=10,zorder=5,ec="k")
    
    # axes.annotate(text='', xy=(scatter_dict["c_aged_cap"]+4,200), xytext=(scatter_dict["c_aged_cap"]-4,200),fontsize=9,
    #                   arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{PE}=?$', xy=(95,240),fontsize=5,ha="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.6))
    # LLI
    axes.annotate(text=r'$\Delta Q_{B}=5.4\,Ah$', xy=(np.max(cap_taycan_aged)-1+3,140),fontsize=5,ha="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.6))
    axes.annotate(text='', xy=(np.max(cap_taycan_aged)+4+3,175), xytext=(np.max(cap_taycan_aged)-4+3,175),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    
    # m phase shift
    axes.annotate(text='', xy=(60,198/2), xytext=(50,187), arrowprops=dict(arrowstyle='<|-', shrinkA=0, shrinkB=0,facecolor="k"))
    axes.annotate(text='M shift', xy=(62,102),horizontalalignment="left",verticalalignment="bottom",fontsize=5)
    return axes

In [ ]:
def add_FOI_tesla_dva(axes,scatter_dict):
    Q_nom = 161
    #anode  
    axes.scatter(scatter_dict["stage_2_init_cap"],scatter_dict["stage_2_init_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMblau.value,s=10,zorder=5,ec="k")
    axes.scatter(scatter_dict["stage_2_aged_cap"],scatter_dict["stage_2_aged_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMorange.value,s=10,zorder=5,ec="k")
    
    axes.annotate(text='', xy=(scatter_dict["stage_2_init_cap"]+4-2,78), xytext=(scatter_dict["stage_2_init_cap"]-4-2,78),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{NE}=1.4\,Ah$', xy=(scatter_dict["stage_2_init_cap"],90),fontsize=5,ha="center")
    
    # LLI
    axes.annotate(text='', xy=(np.max(cap_tesla_aged)+4+7,115), xytext=(np.max(cap_tesla_aged)-4-1,115),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{B}=11.5\,Ah$', xy=(np.max(cap_tesla_aged)-1,130),fontsize=5,ha="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.6))
    return axes

In [ ]:
def add_FOI_teslaY_dva(axes,scatter_dict):
    Q_nom = 46*5
    #anode  
    axes.scatter(scatter_dict["stage_2_init_cap"],scatter_dict["stage_2_init_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMblau.value,s=10,zorder=5,ec="k")
    axes.scatter(scatter_dict["stage_2_aged_cap"],scatter_dict["stage_2_aged_dva"]*Q_nom,marker="d",color=VisualizationConfig.TUMorange.value,s=10,zorder=5,ec="k")
    
    axes.annotate(text='', xy=(scatter_dict["stage_2_init_cap"]+5,150), xytext=(scatter_dict["stage_2_init_cap"]-5,150),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{NE}=2.2\,Ah$', xy=(scatter_dict["stage_2_init_cap"]-2,160),fontsize=5,ha="center")
    # cathode
    axes.scatter(scatter_dict["c_init_cap"],scatter_dict["c_init_dva"]*Q_nom,marker="o",color=VisualizationConfig.TUMblau.value,s=10,zorder=5,ec="k")
    axes.scatter(scatter_dict["c_aged_cap"],scatter_dict["c_aged_dva"]*Q_nom,marker="o",color=VisualizationConfig.TUMorange.value,s=10,zorder=5,ec="k")
    
    axes.annotate(text='', xy=(scatter_dict["c_aged_cap"]+5,140), xytext=(scatter_dict["c_aged_cap"]-5,140),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{PE}=0.2\,Ah$', xy=(scatter_dict["c_aged_cap"]+2,150),fontsize=5,ha="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.6))
    # LLI
    axes.annotate(text='', xy=(np.max(cap_teslaY_aged)+5,100), xytext=(np.max(cap_teslaY_aged)-5-5,100),fontsize=9,
                      arrowprops=dict(arrowstyle='<-', shrinkA=0, shrinkB=0,facecolor="k", mutation_scale=5)) 
    axes.annotate(text=r'$\Delta Q_{B}=0.5\,Ah$', xy=(np.max(cap_teslaY_aged)-3,110),fontsize=5,ha="center",bbox=dict(boxstyle='square,pad=0',fc='white', ec='none', alpha=0.6))
    return axes

In [ ]:
fig = plt.figure(figsize=(cm2inch(VisualizationConfig.textwidth_in_cm.value),
                                                cm2inch(15))) #constrained layout = True does not work with wspace/hspace

gs = fig.add_gridspec(2, 1,height_ratios=[1,1],hspace=0.5)

gs0 = gs[0].subgridspec(2, 2, width_ratios=[1,1],height_ratios=[1,1],wspace=0.25)
gs1 = gs[1].subgridspec(2, 2, width_ratios=[1,1],height_ratios=[1,1],wspace=0.25)
# Volkswagen 
axes1 = fig.add_subplot(gs0[0, 0])
axes1 = plot_pocv_vw(axes1)
axes1 = add_FOI_vw_pocv(axes1)
axes2 = fig.add_subplot(gs0[0, 1])
axes2 = plot_pocv_taycan(axes2)
axes2 = add_FOI_taycan_pocv(axes2)
axes3 = fig.add_subplot(gs0[1, 0])
axes3 = plot_dva_vw(axes3)
axes3 = add_FOI_vw_dva(axes3,scatter_dict_vw)
axes4 = fig.add_subplot(gs0[1, 1])
axes4 = plot_dva_taycan(axes4)
axes4 = add_FOI_taycan_dva(axes4,scatter_dict_taycan)
#align y labels
axes1.yaxis.set_label_coords(-0.12, 0.5)
axes3.yaxis.set_label_coords(-0.105, 0.5) # lower, due to the fraction
axes2.yaxis.set_label_coords(-0.12, 0.5)
axes4.yaxis.set_label_coords(-0.105, 0.5)
## Tesla
axes5 = fig.add_subplot(gs1[0, 0])
axes5 = plot_pocv_tesla(axes5)
axes5 = add_FOI_tesla_pocv(axes5)
axes6 = fig.add_subplot(gs1[0, 1])
axes6 = plot_pocv_teslaY(axes6)
axes6 = add_FOI_teslaY_pocv(axes6)
axes7 = fig.add_subplot(gs1[1, 0])
axes7 = plot_dva_tesla(axes7)
axes7 = add_FOI_tesla_dva(axes7,scatter_dict_tesla)
axes8 = fig.add_subplot(gs1[1, 1])
axes8 = plot_dva_teslaY(axes8)
axes8 = add_FOI_teslaY_dva(axes8,scatter_dict_teslaY)
#align y labels
axes5.yaxis.set_label_coords(-0.12, 0.5)
axes7.yaxis.set_label_coords(-0.105, 0.5) # lower, due to the fraction
axes6.yaxis.set_label_coords(-0.12, 0.5)
axes8.yaxis.set_label_coords(-0.105, 0.5)
#fig.tight_layout()
axes1.text(-0.2, 1.05, 'a)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes1.transAxes)
axes2.text(-0.2, 1.05, r'b)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes2.transAxes)
axes5.text(-0.2, 1.05, r'c)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes5.transAxes)
axes6.text(-0.2, 1.05, r'd)', horizontalalignment='left',fontsize=7,
     verticalalignment='center', transform=axes6.transAxes)

fig.savefig(os.path.join(GeneralConfig.path2figures.value,"10-POCV_DVA_SOH_assessment.pdf"))
fig.savefig(os.path.join(GeneralConfig.path2figures.value,"10-POCV_DVA_SOH_assessment.png"),dpi=300, pad_inches = 0)